In [1]:
import mujoco
import numpy as np

In [2]:
def euler2quat(euler):
    """ Convert Euler Angles to Quaternions.  See rotation.py for notes """
    euler = np.asarray(euler, dtype=np.float64)
    assert euler.shape[-1] == 3, "Invalid shape euler {}".format(euler)

    ai, aj, ak = euler[..., 2] / 2, -euler[..., 1] / 2, euler[..., 0] / 2
    si, sj, sk = np.sin(ai), np.sin(aj), np.sin(ak)
    ci, cj, ck = np.cos(ai), np.cos(aj), np.cos(ak)
    cc, cs = ci * ck, ci * sk
    sc, ss = si * ck, si * sk

    quat = np.empty(euler.shape[:-1] + (4,), dtype=np.float64)
    quat[..., 0] = cj * cc + sj * ss
    quat[..., 3] = cj * sc - sj * cs
    quat[..., 2] = -(cj * ss + sj * cc)
    quat[..., 1] = cj * cs - sj * sc
    return quat


def quat2euler(quat):
    """ Convert Quaternions to Euler Angles.  See rotation.py for notes """
    quat = np.asarray(quat, dtype=np.float64)
    assert quat.shape[-1] == 4, "Invalid shape quat {}".format(quat)

    qw, qx, qy, qz = quat[..., 0], quat[..., 1], quat[..., 2], quat[..., 3]
    ysqr = qy * qy

    t0 = +2.0 * (qw * qx + qy * qz)
    t1 = +1.0 - 2.0 * (qx * qx + ysqr)
    X = np.arctan2(t0, t1)

    t2 = +2.0 * (qw * qy - qz * qx)
    t2 = np.clip(t2, -1.0, 1.0)
    Y = np.arcsin(t2)

    t3 = +2.0 * (qw * qz + qx * qy)
    t4 = +1.0 - 2.0 * (ysqr + qz * qz)
    Z = np.arctan2(t3, t4)

    return np.stack((X, Y, Z), axis=-1)


def radian_dist(start, end):
    """ Compute the angular distance between two angles """
    diff = np.abs(end - start) % (2 * np.pi)
    return np.minimum(diff, 2 * np.pi - diff)

In [3]:
import numpy as np
import robosuite as suite
print(suite.__file__)
import os

os.environ["DISPLAY"] = ":1"    

# create environment instance
env = suite.make(
    env_name="TwoArmACInsertion", 
    env_configuration="single-arm-plaif-ac",  
    robots=["RB3", "RB3"],
    controller_configs={
        "type": "OSC_POSE",
        "input_max": 1,
        "input_min": -1,
        "output_max": [0.05, 0.05, 0.05, 0.5, 0.5, 0.5],
        "output_min": [-0.05, -0.05, -0.05, -0.5, -0.5, -0.5],
        "kp": 150,
        "ki": 1,
        "damping_ratio": 1,
        "impedance_mode": "fixed",
        "kp_limits": [0, 300],
        "damping_ratio_limits": [0, 10],
        "position_limits": None,
        "orientation_limits": None,
        "uncouple_pos_ori": True,
        "control_delta": True,
        "interpolation": None,
        "ramp_ratio": 0.2
    },
    gripper_types=["OnRobotRG2Gripper", "OnRobotRG2Gripper"],  # try with other grippers like "OnRobotRG2Gripper"
    has_renderer=True,
    has_offscreen_renderer=False,
    use_camera_obs=False,
    render_gpu_device_id=0,
)


num_epis = 1000
gripped = False

/home/jhlee/workspace/robosuite/robosuite/__init__.py


In [13]:
obs, done = env.reset(), False
env.render() 

orig_socket_xpos = env._socket_xpos.copy()
orig_plug_xpos = env._plug_xpos.copy()

orig_socket_xpos += [-0.055, -0.02, 0.005]
orig_plug_xpos += [-0.02, 0.03, -0.022]

for i in range(120):
    action = np.random.randn(env.robots[0].dof + env.robots[1].dof) # sample random action
    action[:] = 0
    
    gripper0_to_socket = orig_socket_xpos - env._eef0_xpos
    gripper1_to_plug = orig_plug_xpos - env._eef1_xpos
    
    action[:3] = gripper0_to_socket
    action[6] = 1.0
    action[3] = 0.02 if 0 < i < 100 else 0.0

    action[7:10] = gripper1_to_plug
    action[10] = -0.03 if 0 < i < 100 else 0.0
    action[13] = 1.0
    
    obs, reward, done, info = env.step(action)  # take action in the environment
    if i % 2 == 0:
        env.render()  # render on display
        
    if i == 4:
        input()

for i in range(37):
    action = np.random.randn(env.robots[0].dof + env.robots[1].dof) # sample random action
    action[:] = 0
    
    gripper0_to_socket = orig_socket_xpos - env._eef0_xpos
    gripper1_to_plug = orig_plug_xpos - env._eef1_xpos
    
    action[:3] = gripper0_to_socket
    action[6] = -1.0 if i < 26 else 0.0

    action[7:10] = gripper1_to_plug
    action[13] = -1.0
    
    obs, reward, done, info = env.step(action)  # take action in the environment
    if i % 2 == 0:
        env.render()  # render on display


eef0_target_pos = env._eef0_xpos.copy()
eef1_target_pos = env._eef1_xpos.copy()

eef0_target_pos += [0.17, 0.355, 0.15]
eef1_target_pos += [0.02, -0.26, 0.16]

for i in range(250):
    action = np.random.randn(env.robots[0].dof + env.robots[1].dof) # sample random action
    action[:] = 0
    action[:3] = eef0_target_pos - env._eef0_xpos
    
    plug_to_socket_err = (env._socket_right_hole_end_xpos - env._plug_left_rod_end_xpos + env._socket_left_hole_end_xpos - env._plug_right_rod_end_xpos)/2
    if np.linalg.norm(plug_to_socket_err, 2) > 0.02 and i > 50:
        eef0_yz_align = np.clip(plug_to_socket_err, -(i-50) * 0.1 / 200, (i-50) * 0.1 / 200)
        action[0] -= eef0_yz_align[0]
        action[2] -= eef0_yz_align[2]
    
    action[7:10] = eef1_target_pos - env._eef1_xpos
    if i > 50:
        eef1_yz_align = np.clip(plug_to_socket_err, -(i-50) * 0.1 / 200, (i-50) * 0.1 / 200)
        action[7] += eef1_yz_align[0]
        action[9] += eef1_yz_align[2]
    
    # Level the socket
    action[3] = -quat2euler(obs['socket_quat'])[1] * (i / 250)
    action[4] = env._socket_right_hole_enter_xpos[0] - env._socket_left_hole_enter_xpos[0]
    action[5] = (np.pi/2 - quat2euler(obs['socket_quat'])[0])  * (i / 250)
    
    #print(f"\rSocket Rotations: {quat2euler(obs['socket_quat'])[1]:.6f}, {env._socket_right_hole_enter_xpos[0] - env._socket_left_hole_enter_xpos[0]:.6f}, {quat2euler(obs['socket_quat'])[0]:.6f}", end="", flush=True)
    
    # Level the plug
    action[10] = -quat2euler(obs['plug_quat'])[1]  * (i / 250)
    action[11] = env._plug_left_rod_end_xpos[0] - env._plug_right_rod_end_xpos[0]
    action[12] = (np.pi/2 -quat2euler(obs['plug_quat'])[0])  * (i / 250)
    
    obs, reward, done, info = env.step(action)  # take action in the environment
    if i % 2 == 0:
        env.render()  # render on display

env._socket_right_hole_enter_xpos, env._socket_left_hole_enter_xpos, env._plug_right_rod_end_xpos, env._plug_left_rod_end_xpos

for i in range(200):
    action = np.random.randn(env.robots[0].dof + env.robots[1].dof) # sample random action
    action[:] = 0.
    
    plug_to_socket_err = env._socket_right_hole_end_xpos - env._plug_left_rod_end_xpos + env._socket_left_hole_end_xpos - env._plug_right_rod_end_xpos
    print(f"\r{plug_to_socket_err[0]:.6f}, {plug_to_socket_err[2]:.6f}", end="", flush=True)
    
    if abs(plug_to_socket_err[0]) < 0.003 and abs(plug_to_socket_err[2]) < 0.003:
        action[7:10] = env._socket_right_hole_end_xpos - env._plug_left_rod_end_xpos
    else:
        action[7:10] = plug_to_socket_err
        
    
    #print(f"\rSocket Rotations: {quat2euler(obs['socket_quat'])[1]:.6f}, {env._socket_right_hole_enter_xpos[0] - env._socket_left_hole_enter_xpos[0]:.6f}, {quat2euler(obs['socket_quat'])[0]:.6f}", end="", flush=True)
    
    # Level the plug
    action[10] = -quat2euler(obs['plug_quat'])[1]  * (i / 250)
    action[11] = env._plug_left_rod_end_xpos[0] - env._plug_right_rod_end_xpos[0]
    action[12] = (np.pi/2 -quat2euler(obs['plug_quat'])[0])  * (i / 250)
    
    obs, reward, done, info = env.step(action)  # take action in the environment
    if i % 2 == 0:
        env.render()  # render on display

KeyboardInterrupt: 

In [ ]:
env._socket_right_hole_enter_xpos - env._plug_left_rod_end_xpos

array([ 0.01011304, -0.0200783 ,  0.00076679])

In [ ]:
env._socket_right_hole_enter_xpos - env._plug_left_rod_end_xpos

array([ 0.00288374, -0.02807382,  0.00329946])

In [ ]:
env._plug_right_rod_end_xpos - env._plug_left_rod_end_xpos

array([ 0.01725393, -0.00210556, -0.00467637])

In [ ]:
euler2quat([-0.05048157, -0.25269768, -0.24485742])

array([ 0.98467799, -0.00946533, -0.12808747, -0.11795449])

In [ ]:
radian_dist(np.array([np.pi / 2, 0.0, np.pi]), quat2euler(obs['plug_quat']))

array([0.05048157, 0.25269768, 0.24485742])

In [ ]:
quat2euler(obs['plug_quat'])

array([ 1.48030648, -0.85601697,  2.93845464])

In [ ]:
env._plug_right_rod_end_xpos, env._plug_left_rod_end_xpos

(array([-0.07583044,  0.0151538 ,  0.96599215]),
 array([-0.09311209,  0.01670997,  0.97077994]))

In [ ]:
eef1_target_pos, env._eef1_xpos

(array([0.02496851, 0.13615307, 0.88635005]),
 array([-0.49432118,  0.63467768,  0.53096261]))

In [ ]:
gripper0_to_socket, gripper1_to_plug

(array([-0.04908186, -0.00921243, -0.07480898]),
 array([-0.02198683,  0.00997145, -0.07505174]))

In [ ]:
env._gripper0_to_socket

array([-0.21325114, -0.11081225, -0.10482947])

In [ ]:
(quat2euler(obs['socket_quat'])[2]) - (quat2euler(env._eef0_xquat)[2])

0.02223675777061951

In [ ]:
obs['socket_pos']

array([-0.13263278, -0.33340188,  0.79895678])

In [ ]:
obs['plug_pos']

array([0.00337672, 0.33264879, 0.81853002])

In [ ]:
obs['socket_pos']

array([-0.13263278, -0.33340188,  0.79895678])

In [ ]:
euler2quat([0, 0, 0.01])

array([ 0.9999875 ,  0.        , -0.        ,  0.00499998])

In [ ]:
quat2euler(obs['socket_quat']) - quat2euler(env._eef0_xquat)

array([ 3.18459919, -0.03332776,  0.02223676])

In [ ]:
# get xpos
env.robots[1].robot_model.

SyntaxError: invalid syntax (3235626708.py, line 2)

OrderedDict([('robot0_joint_pos_cos',
              array([-0.99989242,  0.99997616, -0.00379826,  0.99996917,  0.03799365,
                      0.99948689])),
             ('robot0_joint_pos_sin',
              array([-0.01466761, -0.00690534, -0.99999279, -0.00785187, -0.99927798,
                      0.03203055])),
             ('robot0_joint_vel',
              array([ 0.02652421,  0.09659377,  0.28638858,  0.1763355 ,  0.21132374,
                     -0.0228806 ])),
             ('robot0_eef_pos',
              array([ 0.12085572, -0.17620251,  0.92856047])),
             ('robot0_eef_quat',
              array([-0.99986911,  0.00863183, -0.00379517,  0.01314697])),
             ('robot0_gripper_qpos',
              array([ 0.33268597, -0.46095247,  0.30345942, -0.30537047,  0.3799224 ,
                     -0.2610874 ])),
             ('robot0_gripper_qvel',
              array([-0.62562137,  1.85819565, -0.66827422,  0.13675126, -0.70133711,
                      0.55024448])